# Intro

## Data Details

### Columns
'DATE, REPORT_TYPE, SOURCE, AWND, CDSD, CLDD, DSNW, DYHF, DYTS, DailyAverageDryBulbTemperature, DailyAverageStationPressure, DailyAverageWindSpeed, DailyCoolingDegreeDays, DailyDepartureFromNormalAverageTemperature, DailyHeatingDegreeDays, DailyMaximumDryBulbTemperature, DailyMinimumDryBulbTemperature, DailyPeakWindDirection, DailyPeakWindSpeed, DailyPrecipitation, DailySnowDepth, DailySnowfall, DailySustainedWindDirection, DailySustainedWindSpeed, DailyWeather, HDSD, HTDD, HourlyAltimeterSetting, HourlyDewPointTemperature, HourlyDryBulbTemperature, HourlyPrecipitation, HourlyPresentWeatherType, HourlyPressureChange, HourlyPressureTendency, HourlyRelativeHumidity, HourlySeaLevelPressure, HourlySkyConditions, HourlyStationPressure, HourlyVisibility, HourlyWetBulbTemperature, HourlyWindDirection, HourlyWindGustSpeed, HourlyWindSpeed, MonthlyDaysWithGT001Precip, MonthlyDaysWithGT010Precip, MonthlyDaysWithGT32Temp, MonthlyDaysWithGT90Temp, MonthlyDaysWithLT0Temp, MonthlyDaysWithLT32Temp, MonthlyDepartureFromNormalAverageTemperature, MonthlyDepartureFromNormalCoolingDegreeDays, MonthlyDepartureFromNormalHeatingDegreeDays, MonthlyDepartureFromNormalMaximumTemperature, MonthlyDepartureFromNormalMinimumTemperature, MonthlyDepartureFromNormalPrecipitation, MonthlyGreatestPrecip, MonthlyGreatestPrecipDate, MonthlyMaxSeaLevelPressureValue, MonthlyMaxSeaLevelPressureValueDate, MonthlyMaxSeaLevelPressureValueTime, MonthlyMaximumTemperature, MonthlyMeanTemperature, MonthlyMinSeaLevelPressureValue, MonthlyMinSeaLevelPressureValueDate, MonthlyMinSeaLevelPressureValueTime, MonthlyMinimumTemperature, MonthlySeaLevelPressure, MonthlyStationPressure, MonthlyTotalLiquidPrecipitation, NormalsCoolingDegreeDay, NormalsHeatingDegreeDay, REM, REPORT_TYPE.1, SOURCE.1, ShortDurationEndDate005, ShortDurationEndDate010, ShortDurationEndDate015, ShortDurationEndDate020, ShortDurationEndDate030, ShortDurationEndDate045, ShortDurationEndDate060, ShortDurationEndDate080, ShortDurationEndDate100, ShortDurationEndDate120, ShortDurationEndDate150, ShortDurationEndDate180, ShortDurationPrecipitationValue005, ShortDurationPrecipitationValue010, ShortDurationPrecipitationValue015, ShortDurationPrecipitationValue020, ShortDurationPrecipitationValue030, ShortDurationPrecipitationValue045, ShortDurationPrecipitationValue060, ShortDurationPrecipitationValue080, ShortDurationPrecipitationValue100, ShortDurationPrecipitationValue120, ShortDurationPrecipitationValue150, ShortDurationPrecipitationValue180, Sunrise, Sunset, MonthlyInd'

## Analysis Code

In [476]:
import pandas as pd
import numpy as np
from plotnine import *
import re
import plotly.express as px
import datetime as dt
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)


In [241]:
def DropNaCols(df):
    '''given a dataframe, drop all the columns with nothing but NaN'''
    naColList = []
    for ele in df.columns:
        uniqueVals = list(df[ele].unique())
        if len(uniqueVals) == 1:
            naColList.append(ele)
    return(df.drop(columns=naColList))

In [306]:
def ReadingType(df):
    '''indicates whether the given row provides a monthly, daily, or hourly reading'''
    
    #get the different column types into separate lists
    monthlyCols = [ele for ele in list(df.columns) if 'month' in ele.lower()]
    dailyCols = [ele for ele in list(df.columns) if 'dai' in ele.lower()]
    hourlyCols = [ele for ele in list(df.columns) if 'hour' in ele.lower()]

    #create columns indicating whether a row contains monthly, daily, or hourly readings
    boolMask_monthly = ~df[monthlyCols].isna()
    df['monthlyInd'] = boolMask_monthly.sum(axis=1)

    boolMask_daily = ~df[dailyCols].isna()
    df['dailyInd'] = boolMask_daily.sum(axis=1)

    boolMask_hourly = ~df[hourlyCols].isna()
    df['hourlyInd'] = boolMask_hourly.sum(axis=1)

    #if a row has more than one non-NaN value in a monthly column, the reading is monthly, else if more than one non-NaN value in a daily column, the reading is daily, else if more than one non-Nan value in an hourly column, the reading is hourly else indicate no valid readings for the row
    df['ReadingType'] = ['monthly' if df.iloc[ele]['monthlyInd'] > 0 else 'daily' if df.iloc[ele]['dailyInd'] > 0 else 'hourly' if df.iloc[ele]['hourlyInd'] > 0 else 'noValidReading' for ele in range(df.shape[0])]

    return df

In [243]:
def SplitDataframes(df):
    '''given a dataframe of labeled monthly, daily, and hourly readings, split the dataframe by those labels into component dataframes and load those to a dictionary labeled according to reading type'''
    dfDict = {}
    for ele in df['ReadingType'].unique():
        dfDict[ele] = DropNaCols(df[df['ReadingType'] == ele]).reset_index(drop=True)
    return dfDict

In [479]:
def CleanWeatherDF(df):
    '''given a dataframe of local weather data, do some cosmetic cleaning including...
    1. remove uppercase from column names and add underscores between words
    2. convert date column to datetime'''

    #1. remove uppercase from column names and add underscores between words
    new_column_names = []
    for ele in list(df.columns):
        if ele.isupper():
            new_column_names.append(re.sub('([.])','_',ele.lower()))
        else:
            new_column_names.append(re.sub('([.])','_',re.sub('(?<!^)(?=[A-Z])', '_',ele).lower()))
    df.columns = new_column_names

    # convert date column to datetime
    df['date'] = pd.to_datetime(df['date'])

    return df

In [428]:
#clean up numerican columns, converting strings to digits, removing unnecessary strings, converting to floats
def CleanHourlyColumns(df):
    '''given a dataframe of hourly weather values, clean up the column values'''
    numerical_columns = ['hourly_altimeter_setting','hourly_dew_point_temperature','hourly_dry_bulb_temperature','hourly_precipitation','hourly_pressure_change','hourly_pressure_tendency','hourly_relative_humidity','hourly_sea_level_pressure','hourly_station_pressure','hourly_visibility','hourly_wet_bulb_temperature','hourly_wind_direction','hourly_wind_gust_speed','hourly_wind_speed']

    for ele in numerical_columns:
        df[ele] = df[ele].replace('[A-Za-z*]','',regex=True).replace('',0).astype(float).fillna(0)

    return df

In [480]:
#read in data
df = pd.read_csv('3063831.csv')

#add reading type column
df = ReadingType(df)

#clean up data
df = DropNaCols(df)

#next, put dataframe into separate dataframes depending on the type of reading
dfDict = SplitDataframes(df)

#clean up dataframes
for ele in dfDict.values():
    ele = CleanWeatherDF(ele)

#for numerical columns in the hourly data, make them truly numerical
dfDict['hourly'] = CleanHourlyColumns(dfDict['hourly'])

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (21,27,28,29,30,31,36,37,38,42,43,49,50,52,54,57,59,60,61,62,64,115,118,120) have mixed types.Specify dtype option on import or set low_memory=False.



In [486]:
df = dfDict['hourly']

In [502]:
df.columns

Index(['date', 'report_type', 'source', 'hourly_altimeter_setting',
       'hourly_dew_point_temperature', 'hourly_dry_bulb_temperature',
       'hourly_precipitation', 'hourly_present_weather_type',
       'hourly_pressure_change', 'hourly_pressure_tendency',
       'hourly_relative_humidity', 'hourly_sea_level_pressure',
       'hourly_sky_conditions', 'hourly_station_pressure', 'hourly_visibility',
       'hourly_wet_bulb_temperature', 'hourly_wind_direction',
       'hourly_wind_gust_speed', 'hourly_wind_speed', 'rem', 'report_type_1',
       'source_1', 'hourly_ind'],
      dtype='object')

In [503]:
X = df['hourly_dew_point_temperature']
y = df['hourly_dry_bulb_temperature']

X_train, X_test, y_train, y_split = train_test_split(X, y, test_size=.2, random_state=32)

In [505]:
y_train

35420    38.0
32878    67.0
38189    42.0
36928    36.0
12180    57.0
         ... 
19527    79.0
24828    24.0
20414    81.0
9526     55.0
10967    32.0
Name: hourly_dry_bulb_temperature, Length: 33663, dtype: float64